### Submodular Gradient Ascent

In [1]:
from comblearn.nn import DSF
import torch

In [2]:
device = 'cuda'

In [3]:
n = 10
mo = 50
dsf = DSF(n, 1, mo, [3, 4, 3], 20).to(device)

In [4]:
x = torch.tensor([1, 0, 1, 0, 0, 0, 0, 0, 0, 1]).float().to(device)
dsf(x)

tensor([42.8354], device='cuda:0', grad_fn=<MinimumBackward0>)

In [5]:
list(dsf.parameters())[0]

Parameter containing:
tensor([[0.6361, 0.5304, 3.3328],
        [1.1875, 0.6348, 0.8923],
        [1.6668, 0.6573, 0.4416],
        [0.6120, 0.0719, 0.5457],
        [0.3169, 0.9287, 0.3035],
        [0.7851, 0.3455, 2.2994],
        [0.7255, 0.1071, 0.5849],
        [0.8807, 1.2415, 1.1041],
        [0.8398, 0.5560, 0.1176],
        [1.3233, 0.5000, 0.1965]], device='cuda:0', requires_grad=True)

In [6]:
from torch import nn

class DSFWrapper(nn.Module):
    def __init__(self, n, dsf):
        super().__init__()
        self.weights = nn.Parameter(torch.zeros((n, ))).float()
        self.submodular = dsf

    def forward(self, x):
        mask = (x > 0).float()
        return self.submodular(mask * self.weights)

In [7]:
from torch.optim import SGD

eps = 0.05
T = int((n / eps) ** 2)
lr = 2e-5

wrapper = DSFWrapper(n, dsf).to(device)

In [8]:
list(wrapper.parameters())

[Parameter containing:
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0',
        requires_grad=True),
 Parameter containing:
 tensor([[0.6361, 0.5304, 3.3328],
         [1.1875, 0.6348, 0.8923],
         [1.6668, 0.6573, 0.4416],
         [0.6120, 0.0719, 0.5457],
         [0.3169, 0.9287, 0.3035],
         [0.7851, 0.3455, 2.2994],
         [0.7255, 0.1071, 0.5849],
         [0.8807, 1.2415, 1.1041],
         [0.8398, 0.5560, 0.1176],
         [1.3233, 0.5000, 0.1965]], device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([0., 0., 0.], device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([[1.0184, 0.3925, 2.3786, 0.8904],
         [0.2970, 0.8281, 0.9875, 1.1196],
         [0.2599, 0.9083, 1.7407, 0.5452]], device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([0., 0., 0., 0.], device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([[0.3431, 1.2957, 0.8684],
         [0.7636, 0.8492, 2.2784],
         [0.520

In [9]:
batch_size = 10
last_pred = -1.0

for i in range(T // batch_size):
    print(f"Step {i}/{T // batch_size}")
    
    m = torch.ones((batch_size, n)).float().to(device)
    pred = wrapper(m).mean()
    print(pred)
    pred.backward()
    g = wrapper.weights.grad
    
    # if pred == last_pred:
    #     break
    # else:
    #     last_pred = pred

    print('G:', g)
    with torch.no_grad():
        wrapper.weights.add_(lr * g)
        wrapper.weights.abs_()
        wrapper.weights.clamp_max_(1)


    print('W:', list(wrapper.parameters())[0])
    # print('P:', list(dsf.parameters())[0][0])
    
# dsf.requires_grad_(True)

Step 0/4000
tensor(0., device='cuda:0', grad_fn=<MeanBackward0>)
G: tensor([28.8838, 18.5760, 19.6271,  8.5894,  9.8836, 22.4982,  9.9261, 21.1221,
        10.5725, 14.4911], device='cuda:0')
W: Parameter containing:
tensor([0.0006, 0.0004, 0.0004, 0.0002, 0.0002, 0.0004, 0.0002, 0.0004, 0.0002,
        0.0003], device='cuda:0', requires_grad=True)
Step 1/4000
tensor(0.0622, device='cuda:0', grad_fn=<MeanBackward0>)
G: tensor([57.7676, 37.1520, 39.2543, 17.1788, 19.7672, 44.9964, 19.8522, 42.2442,
        21.1451, 28.9822], device='cuda:0')
W: Parameter containing:
tensor([0.0017, 0.0011, 0.0012, 0.0005, 0.0006, 0.0013, 0.0006, 0.0013, 0.0006,
        0.0009], device='cuda:0', requires_grad=True)
Step 2/4000
tensor(0.1865, device='cuda:0', grad_fn=<MeanBackward0>)
G: tensor([86.6514, 55.7280, 58.8814, 25.7682, 29.6508, 67.4946, 29.7783, 63.3664,
        31.7176, 43.4733], device='cuda:0')
W: Parameter containing:
tensor([0.0035, 0.0022, 0.0024, 0.0010, 0.0012, 0.0027, 0.0012, 0.0025, 0

KeyboardInterrupt: 

In [19]:
wrapper.weights.clone().detach()

tensor([0.1280, 0.4939, 0.5340, 0.3671, 0.6000, 0.3416, 0.2466, 0.2304, 0.2477,
        0.4507], device='cuda:0')

In [20]:
y = torch.zeros((3, 4)).to(device)
y

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], device='cuda:0')

In [22]:
y[:, 2] = torch.tensor([1, 2, 3])
y

tensor([[0., 0., 1., 0.],
        [0., 0., 2., 0.],
        [0., 0., 3., 0.]], device='cuda:0')

In [23]:
eval("torch.optim.SGD")

torch.optim.sgd.SGD

In [45]:
import yaml
import os

print(os.getcwd())

with open("../config/config_rg.yaml") as fp:
    cfg = yaml.load(fp, Loader=yaml.FullLoader)

/home/mohammad_hosseini/auction/notebooks


In [49]:
import comblearn
eval(cfg['auction']['bidders'][0]['cls'])

comblearn.data.value.DSFValueFunction

In [35]:
from comblearn.data import DSFValueFunction

eval('DSFValueFunction(items, 100, [2, 3, 2], 500)')

NameError: name 'items' is not defined

In [41]:
from tensordict.tensordict import TensorDict

In [43]:
t = TensorDict({'A': torch.tensor([1, 2, 3]), 'B': torch.tensor([4, 5, 6])}, batch_size=1)

RuntimeError: batch dimension mismatch, got self.batch_size=torch.Size([1]) and value.shape[:self.batch_dims]=torch.Size([3]) with value tensor([1, 2, 3])

In [44]:
{b: i for b, i in zip(range(3), range(3))}

{0: 0, 1: 1, 2: 2}